# DOTA 使用deeplab做15类的异常检测, ResNet50对error_map分类

## 输出3*15通道 [x0,x1,...,x14], 查看wx 与  [x0,x1,...,x14]之间的距离

In [1]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.autograd import Variable
import os
import matplotlib.pyplot as plt
import datetime
import time
import numpy as np

# 载入数据

In [2]:
#data_dir = r"D:/datasets/ShipVSField"
data_dir = r"D:\datasets\dota\Slices"
data_transform = {x:transforms.Compose([transforms.Resize([256,256]),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)])
                                      for x in ["train"] } #, "valid"

# 删除隐藏文件夹 .ipynb_checkpoints
for x in ["train", "valid"]:
    if os.path.exists( os.path.join(data_dir,x,".ipynb_checkpoints") ):
        print(u"Delete ".format(os.path.join(data_dir,x,".ipynb_checkpoints") ))
        os.removedirs(os.path.join(data_dir,x,".ipynb_checkpoints"))
    
image_datasets = {x:datasets.ImageFolder(root = os.path.join(data_dir,x),
                                        transform = data_transform[x]) for x in ["train"]} #, "valid"

dataloader = {x:torch.utils.data.DataLoader(dataset = image_datasets[x],
                                           batch_size = 8, #16,
                                           shuffle = True) for x in ["train"]}#, "valid"

In [3]:
X_example,y_example = next(iter( dataloader["train"] ) )
example_classes = image_datasets["train"].classes
index_classes = image_datasets["train"].class_to_idx
cls_num = len(index_classes)

print(index_classes)

{'baseball-diamond': 0, 'basketball-court': 1, 'bridge': 2, 'ground-track-field': 3, 'harbor': 4, 'helicopter': 5, 'large-vehicle': 6, 'plane': 7, 'roundabout': 8, 'ship': 9, 'small-vehicle': 10, 'soccer-ball-field': 11, 'storage-tank': 12, 'swimming-pool': 13, 'tennis-court': 14}


# 配置模型

In [5]:
# deeplabv3_resnet50
num_classes = 3 * 15 # 15类
#model_fcn = torchvision.models.segmentation.fcn_resnet50(pretrained=False, progress=True, num_classes=num_classes, aux_loss=None)
model_deeplab = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=False, progress=True, 
                                                                   num_classes=num_classes, aux_loss=None)

In [6]:
# Resnet50 分类
model_res = torchvision.models.resnet50(pretrained= False)
state_dict = torch.load("resnet50-19c8e357.pth")
model_res.load_state_dict(state_dict)

# 修改输入输出通道数
model_res.fc = torch.nn.Linear(2048, 15)
model_res.conv1 = torch.nn.Conv2d(3*15,64, kernel_size=7,padding=3,bias=False)

In [7]:
#加载模型参数
model_res.load_state_dict(torch.load('DOAT_Slices_deeplab-ResNet50_model_last.pth')['model_res'])
model_deeplab.load_state_dict(torch.load('DOAT_Slices_deeplab-ResNet50_model_last.pth')['model_deeplab'])

<All keys matched successfully>

In [8]:
y_fcn = model_deeplab(X_example )['out'] # * masked_gaussian
print(y_fcn.shape)

cls_num = len(index_classes)

Classes = np.random.randint(low=0, high=cls_num, size=len(X_example))
y_class = torch.cat( [y_fcn[i:i+1,np.array(range(0,3))+3*Classes[i],:,:] for i in range(len(Classes))], axis=0)

torch.pow(X_example - y_class, 2).sum(), 

#torch.tensor(X_example.shape).prod(dim=0)

torch.Size([8, 45, 256, 256])


(tensor(192299.3906, grad_fn=<SumBackward0>),)

In [8]:
error_map = torch.pow(torch.cat([X_example]*cls_num, axis=1) - y_fcn, 2)
print(error_map.shape)
y_res = model_res(error_map)
y_res.shape

torch.Size([8, 45, 256, 256])


torch.Size([8, 15])

# 损失和优化函数

In [8]:
class SquaredErrorLoss(torch.nn.Module):
    def __init__(self):
        super(SquaredErrorLoss, self).__init__()
        
    def forward(self, X, Y, Classes):
        input_shape = torch.tensor(X.size())
        n = X.shape[1]
        index = torch.tensor((range(0,n))).to(Classes.device)
        Y_class = torch.cat( [Y[i:i+1,index+n*Classes[i],:,:] for i in range(len(Classes))], axis=0)
        Error = torch.pow(X - Y_class, 2).sum() / input_shape.prod()
        return Error
    
# FCN 损失和优化函数
loss_fcn = SquaredErrorLoss()

# ResNet损失和优化函数
loss_res = torch.nn.CrossEntropyLoss()

import itertools
optimizer = torch.optim.Adam(itertools.chain(model_deeplab.parameters(),model_res.parameters()), lr=0.00001)

In [ ]:
Classes = torch.tensor( np.random.randint(low=0, high=cls_num, size=len(X_example)) )
print(Classes)

loss_fcn(X_example,y_fcn, Classes)

# 配置GPU，开始进行训练

In [9]:
import collections
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.nn.functional as F
import time
import pdb

In [12]:
try:
    del X_example
    del y
    del y_class
    del error_map
except:
    1

In [25]:
torch.cuda.empty_cache()

In [26]:
# 配置GPU，开始进行训练
Use_gpu = torch.cuda.is_available()

if Use_gpu:
    print( torch.cuda.get_device_properties(device = 0) )
    model_deeplab = model_deeplab.cuda()
    model_res = model_res.cuda()
    
# 配置多GPU
if torch.cuda.device_count() > 1:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model_deeplab = torch.nn.DataParallel(model_deeplab)
    model_res = torch.nn.DataParallel(model_res)
    
masked_gaussian = masked_gaussian.to(next(model_deeplab.parameters()).device)

_CudaDeviceProperties(name='NVIDIA GeForce GTX 1080', major=6, minor=1, total_memory=8192MB, multi_processor_count=20)
Let's use 2 GPUs!


In [27]:
epoch_n = 100
time_open = time.time()
f = open('Loss_log.txt', 'a+')
f.write("\nDOAT_Slices_deeplabv3 begin\n")

for epoch in range(20, epoch_n):  ##### 20
    print("Epoch {}/{}".format(epoch, epoch_n - 1))
    print("-"*10)
    
    for phase in ["train"]: # "valid"
        if phase == "train":
            print("Training...")
            model_deeplab.train(True)
            model_res.train(True)
        else:
            print("Valid...")
            model_deeplab.train(False)
            model_res.train(False) # 把BatchNormalization和DropOut固定住，不会取平均，而是用训练好的值
            
        running_loss = 0.0
        running_corrects = 0.0
        
        for batch, data in enumerate(dataloader[phase], start = 1): #组合为一个索引序列
            X, y = data
            if Use_gpu:
                X, y = Variable(X.cuda()), Variable(y.cuda())
            else:
                X, y = Variable(X), Variable(y)
            
            # 添加高斯掩膜
            y_fcn = model_deeplab.forward(X * masked_gaussian.to(X.device))['out']
            
            error_map = torch.pow(torch.cat([X]*cls_num, axis=1) - y_fcn, 2)
            y_pred = model_res.forward(error_map)
            
            
            _, pred = torch.max(y_pred.data, 1)
            
            loss = loss_fcn(X, y_fcn, y) + loss_res(y_pred, y)
            
            # 对参数梯度的归零
            optimizer.zero_grad()
            if phase == "train":
                # 计算反向传播梯度值 P167
                loss.backward() 
                # 对节点的参数进行梯度更新 P169
                optimizer.step()
                
            running_loss += float(loss.data)
            running_corrects += float(torch.sum(pred == y.data))
            
            if batch%500 == 0 and phase == "train":
                print("Batch {}, Train Loss:{:.4f}, Train ACC:{:.4f}"
                      .format(batch, running_loss/batch, 100*running_corrects/batch/len(pred)))
                print("Current time {:.2f} min".format((time.time() - time_open)/60))
            
        epoch_loss = running_loss*16/len(image_datasets[phase])
        epoch_acc = 100*running_corrects/len(image_datasets[phase])
        
        print("{} Loss:{:.4f}  Acc:{:.4f}%".format(phase, epoch_loss, epoch_acc))
        time_end = time.time() - time_open
        print("Finish time {:.2f} min".format(time_end/60))
        
        if torch.cuda.device_count() > 1:
            torch.save({'model_res':model_res.module.state_dict(),
                        'model_deeplab':model_deeplab.state_dict()}, # module.
                        'DOAT_Slices_deeplab-ResNet50_epoch_{}.pth'.format(epoch))
        else:
            torch.save({'model_res':model_res.state_dict(), 
                        'model_deeplab':model_deeplab.state_dict()},
                        'DOAT_Slices_deeplab-ResNet50_epoch_{}.pth'.format(epoch))
    
        
# 清除显卡缓存
torch.cuda.empty_cache()
f.close()
if torch.cuda.device_count() > 1:
    torch.save({'model_res':model_res.module.state_dict(),
                'model_deeplab':model_deeplab.state_dict()}, #.module
                'DOAT_Slices_deeplab-ResNet50_model_last.pth')
else:
    torch.save({'model_res':model_res.state_dict(), 
                'model_deeplab':model_deeplab.state_dict()},
                'DOAT_Slices_deeplab-ResNet50_model_last.pth')

Epoch 20/99
----------
Training...


C:\ProgramData\Anaconda3\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')


Batch 500, Train Loss:0.0306, Train ACC:99.7750
Current time 8.21 min
Batch 1000, Train Loss:0.0297, Train ACC:99.7750
Current time 16.32 min
Batch 1500, Train Loss:0.0310, Train ACC:99.7500
Current time 24.42 min
Batch 2000, Train Loss:0.0314, Train ACC:99.7125
Current time 32.53 min
Batch 2500, Train Loss:0.0307, Train ACC:99.7400
Current time 40.65 min
Batch 3000, Train Loss:0.0307, Train ACC:99.7292
Current time 48.74 min
Batch 3500, Train Loss:0.0304, Train ACC:99.7429
Current time 56.88 min
Batch 4000, Train Loss:0.0311, Train ACC:99.7250
Current time 64.98 min
Batch 4500, Train Loss:0.0312, Train ACC:99.7361
Current time 73.07 min
Batch 5000, Train Loss:0.0317, Train ACC:99.7350
Current time 81.17 min
Batch 5500, Train Loss:0.0318, Train ACC:99.7386
Current time 89.27 min
Batch 6000, Train Loss:0.0321, Train ACC:99.7313
Current time 97.37 min
Batch 6500, Train Loss:0.0321, Train ACC:99.7385
Current time 105.49 min
Batch 7000, Train Loss:0.0319, Train ACC:99.7429
Current time 113

Batch 8500, Train Loss:0.0314, Train ACC:99.7324
Current time 916.68 min
Batch 9000, Train Loss:0.0314, Train ACC:99.7306
Current time 924.79 min
Batch 9500, Train Loss:0.0312, Train ACC:99.7329
Current time 932.91 min
Batch 10000, Train Loss:0.0310, Train ACC:99.7362
Current time 941.04 min
Batch 10500, Train Loss:0.0310, Train ACC:99.7369
Current time 949.16 min
Batch 11000, Train Loss:0.0309, Train ACC:99.7375
Current time 957.29 min
Batch 11500, Train Loss:0.0308, Train ACC:99.7370
Current time 965.40 min
train Loss:0.0617  Acc:99.7317%
Finish time 972.37 min
Epoch 25/99
----------
Training...
Batch 500, Train Loss:0.0255, Train ACC:99.9000
Current time 980.51 min
Batch 1000, Train Loss:0.0262, Train ACC:99.8625
Current time 988.62 min
Batch 1500, Train Loss:0.0276, Train ACC:99.8250
Current time 996.74 min
Batch 2000, Train Loss:0.0286, Train ACC:99.8063
Current time 1004.90 min
Batch 2500, Train Loss:0.0286, Train ACC:99.8050
Current time 1013.01 min
Batch 3000, Train Loss:0.0303

Batch 3500, Train Loss:0.0273, Train ACC:99.8036
Current time 1807.38 min
Batch 4000, Train Loss:0.0276, Train ACC:99.7906
Current time 1815.69 min
Batch 4500, Train Loss:0.0281, Train ACC:99.7639
Current time 1824.02 min
Batch 5000, Train Loss:0.0277, Train ACC:99.7825
Current time 1832.36 min
Batch 5500, Train Loss:0.0276, Train ACC:99.7841
Current time 1840.64 min
Batch 6000, Train Loss:0.0276, Train ACC:99.7812
Current time 1848.96 min
Batch 6500, Train Loss:0.0275, Train ACC:99.7885
Current time 1857.26 min
Batch 7000, Train Loss:0.0274, Train ACC:99.7875
Current time 1865.60 min
Batch 7500, Train Loss:0.0273, Train ACC:99.7900
Current time 1873.92 min
Batch 8000, Train Loss:0.0278, Train ACC:99.7781
Current time 1882.21 min
Batch 8500, Train Loss:0.0280, Train ACC:99.7765
Current time 1890.54 min
Batch 9000, Train Loss:0.0278, Train ACC:99.7792
Current time 1898.84 min
Batch 9500, Train Loss:0.0276, Train ACC:99.7868
Current time 1907.14 min
Batch 10000, Train Loss:0.0275, Train 

...

Batch 7500, Train Loss:0.0201, Train ACC:99.9100
Current time 11463.21 min
Batch 8000, Train Loss:0.0200, Train ACC:99.9141
Current time 11471.40 min
Batch 8500, Train Loss:0.0199, Train ACC:99.9176
Current time 11479.59 min
Batch 9000, Train Loss:0.0200, Train ACC:99.9139
Current time 11487.79 min
Batch 9500, Train Loss:0.0200, Train ACC:99.9118
Current time 11495.98 min
Batch 10000, Train Loss:0.0199, Train ACC:99.9150
Current time 11504.16 min
Batch 10500, Train Loss:0.0199, Train ACC:99.9155
Current time 11512.36 min
Batch 11000, Train Loss:0.0198, Train ACC:99.9159
Current time 11520.53 min
Batch 11500, Train Loss:0.0198, Train ACC:99.9163
Current time 11528.70 min
train Loss:0.0399  Acc:99.9130%
Finish time 11535.72 min
Epoch 80/99
----------
Training...
Batch 500, Train Loss:0.0195, Train ACC:99.9500
Current time 11543.77 min
Batch 1000, Train Loss:0.0186, Train ACC:99.9750
Current time 11551.85 min
Batch 1500, Train Loss:0.0181, Train ACC:99.9833
Current time 11559.89 min
Batch

Batch 1500, Train Loss:0.0191, Train ACC:99.9167
Current time 12321.59 min
Batch 2000, Train Loss:0.0201, Train ACC:99.9125
Current time 12329.74 min
Batch 2500, Train Loss:0.0198, Train ACC:99.9150
Current time 12337.91 min
Batch 3000, Train Loss:0.0193, Train ACC:99.9292
Current time 12346.10 min
Batch 3500, Train Loss:0.0196, Train ACC:99.9214
Current time 12354.26 min
Batch 4000, Train Loss:0.0203, Train ACC:99.8969
Current time 12362.43 min
Batch 4500, Train Loss:0.0203, Train ACC:99.8889
Current time 12370.60 min
Batch 5000, Train Loss:0.0200, Train ACC:99.8975
Current time 12378.79 min
Batch 5500, Train Loss:0.0198, Train ACC:99.9023
Current time 12386.94 min
Batch 6000, Train Loss:0.0200, Train ACC:99.9021
Current time 12395.11 min
Batch 6500, Train Loss:0.0204, Train ACC:99.8904
Current time 12403.27 min
Batch 7000, Train Loss:0.0205, Train ACC:99.8893
Current time 12411.47 min
Batch 7500, Train Loss:0.0207, Train ACC:99.8867
Current time 12419.66 min
Batch 8000, Train Loss:0.

Batch 8000, Train Loss:0.0193, Train ACC:99.9203
Current time 13188.88 min
Batch 8500, Train Loss:0.0192, Train ACC:99.9221
Current time 13196.73 min
Batch 9000, Train Loss:0.0191, Train ACC:99.9236
Current time 13204.58 min
Batch 9500, Train Loss:0.0194, Train ACC:99.9184
Current time 13212.47 min
Batch 10000, Train Loss:0.0197, Train ACC:99.9112
Current time 13220.58 min
Batch 10500, Train Loss:0.0198, Train ACC:99.9107
Current time 13228.73 min
Batch 11000, Train Loss:0.0200, Train ACC:99.9068
Current time 13236.85 min
Batch 11500, Train Loss:0.0199, Train ACC:99.9087
Current time 13244.99 min
train Loss:0.0398  Acc:99.9109%
Finish time 13251.97 min
Epoch 89/99
----------
Training...
Batch 500, Train Loss:0.0202, Train ACC:99.9750
Current time 13260.09 min
Batch 1000, Train Loss:0.0188, Train ACC:99.9625
Current time 13268.21 min
Batch 1500, Train Loss:0.0179, Train ACC:99.9750
Current time 13276.25 min
Batch 2000, Train Loss:0.0177, Train ACC:99.9750
Current time 13284.34 min
Batch

Batch 2000, Train Loss:0.0187, Train ACC:99.9188
Current time 14052.32 min
Batch 2500, Train Loss:0.0186, Train ACC:99.9250
Current time 14060.22 min
Batch 3000, Train Loss:0.0183, Train ACC:99.9375
Current time 14068.12 min
Batch 3500, Train Loss:0.0182, Train ACC:99.9393
Current time 14076.04 min
Batch 4000, Train Loss:0.0182, Train ACC:99.9375
Current time 14083.93 min
Batch 4500, Train Loss:0.0182, Train ACC:99.9361
Current time 14091.80 min
Batch 5000, Train Loss:0.0184, Train ACC:99.9250
Current time 14099.69 min
Batch 5500, Train Loss:0.0184, Train ACC:99.9227
Current time 14107.61 min
Batch 6000, Train Loss:0.0188, Train ACC:99.9188
Current time 14115.49 min
Batch 6500, Train Loss:0.0187, Train ACC:99.9192
Current time 14123.37 min
Batch 7000, Train Loss:0.0189, Train ACC:99.9125
Current time 14131.30 min
Batch 7500, Train Loss:0.0191, Train ACC:99.9083
Current time 14139.18 min
Batch 8000, Train Loss:0.0192, Train ACC:99.9078
Current time 14147.10 min
Batch 8500, Train Loss:0.

Batch 8500, Train Loss:0.0189, Train ACC:99.9412
Current time 14918.07 min
Batch 9000, Train Loss:0.0189, Train ACC:99.9403
Current time 14925.94 min
Batch 9500, Train Loss:0.0190, Train ACC:99.9408
Current time 14933.80 min
Batch 10000, Train Loss:0.0193, Train ACC:99.9313
Current time 14941.69 min
Batch 10500, Train Loss:0.0192, Train ACC:99.9333
Current time 14949.53 min
Batch 11000, Train Loss:0.0194, Train ACC:99.9295
Current time 14957.45 min
Batch 11500, Train Loss:0.0194, Train ACC:99.9315
Current time 14965.32 min
train Loss:0.0387  Acc:99.9340%
Finish time 14972.08 min
Epoch 98/99
----------
Training...
Batch 500, Train Loss:0.0208, Train ACC:99.8750
Current time 14979.87 min
Batch 1000, Train Loss:0.0193, Train ACC:99.9000
Current time 14987.65 min
Batch 1500, Train Loss:0.0184, Train ACC:99.9250
Current time 14995.43 min
Batch 2000, Train Loss:0.0202, Train ACC:99.8937
Current time 15003.22 min
Batch 2500, Train Loss:0.0197, Train ACC:99.9100
Current time 15011.02 min
Batch

In [28]:
if torch.cuda.device_count() > 1:
    torch.save({'model_res':model_res.module.state_dict(),
                'model_deeplab':model_deeplab.state_dict()}, #.module
                'DOAT_Slices_deeplab-ResNet50_model_last.pth')
else:
    torch.save({'model_res':model_res.state_dict(), 
                'model_deeplab':model_deeplab.state_dict()},
                'DOAT_Slices_deeplab-ResNet50_model_last.pth')

In [19]:
print("休眠",datetime.datetime.now())
os.system("rundll32.exe powrprof.dll,SetSuspendState 0,1,0")
os.system("rundll32.exe powrprof.dll,SetSuspendState")
print("休眠",datetime.datetime.now())

休眠 2022-07-26 17:52:10.801712
休眠 2022-07-27 11:04:16.536572


# Valid

In [10]:
# 配置GPU，开始进行训练
Use_gpu = torch.cuda.is_available()

if Use_gpu:
    print( torch.cuda.get_device_properties(device = 0) )
    model_deeplab = model_deeplab.cuda()
    model_res = model_res.cuda()
    
# 配置多GPU
if torch.cuda.device_count() > 1:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'  # 
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model_deeplab = torch.nn.DataParallel(model_deeplab)
    model_res = torch.nn.DataParallel(model_res)
    
masked_gaussian = masked_gaussian.to(next(model_deeplab.parameters()).device)

_CudaDeviceProperties(name='NVIDIA GeForce GTX 1080', major=6, minor=1, total_memory=8192MB, multi_processor_count=20)
Let's use 2 GPUs!


In [11]:
time_open = time.time()
print("Valid...")

phase = "train"

model_deeplab.train(False) # 把BatchNormalization和DropOut固定住，不会取平均，而是用训练好的值
model_res.train(False)         
    
running_loss = 0.0
running_corrects = 0.0
predict = [[] for i in range(len(example_classes))]

for batch, data in enumerate(dataloader[phase], start = 1): #组合为一个索引序列
    X, y = data
    if Use_gpu:
        X, y = Variable(X.cuda()), Variable(y.cuda())
    else:
        X, y = Variable(X), Variable(y)

    # 添加高斯掩膜
    y_fcn = model_deeplab.forward(X * masked_gaussian.to(X.device))['out']

    error_map = torch.pow(torch.cat([X]*cls_num, axis=1) - y_fcn, 2)
    y_pred = model_res.forward(error_map)
    
    _, pred = torch.max(y_pred.data, 1)

    loss = loss_fcn(X, y_fcn, y) + loss_res(y_pred, y)
    
    # 记录预测结果
    for i in range(len(y)):
        predict[y[i]].append( y_pred[i].cpu().detach().numpy() )  #.argmax(dim=1)
    
    running_loss += float(loss.data)
    running_corrects += float(torch.sum(pred == y.data))

    if batch%500 == 0:# and phase == "train":
        print("Batch {}, Train Loss:{:.4f}, Train ACC:{:.4f}"
              .format(batch, running_loss/batch, 100*running_corrects/batch/len(pred)))
        print("Current time {:.2f} min".format((time.time() - time_open)/60))

epoch_loss = running_loss*16/len(image_datasets[phase])
epoch_acc = 100*running_corrects/len(image_datasets[phase])

print("{} Loss:{:.4f}  Acc:{:.4f}%".format(phase, epoch_loss, epoch_acc))
time_end = time.time() - time_open
print("Finish time {:.2f} min".format(time_end/60))

Valid...
Batch 500, Train Loss:0.0168, Train ACC:99.9500
Current time 2.46 min
Batch 1000, Train Loss:0.0162, Train ACC:99.9750
Current time 4.84 min
Batch 1500, Train Loss:0.0160, Train ACC:99.9750
Current time 7.23 min
Batch 2000, Train Loss:0.0159, Train ACC:99.9750
Current time 9.64 min
Batch 2500, Train Loss:0.0164, Train ACC:99.9600
Current time 12.04 min
Batch 3000, Train Loss:0.0165, Train ACC:99.9500
Current time 14.44 min
Batch 3500, Train Loss:0.0175, Train ACC:99.9357
Current time 16.85 min
Batch 4000, Train Loss:0.0172, Train ACC:99.9437
Current time 19.26 min
Batch 4500, Train Loss:0.0172, Train ACC:99.9389
Current time 21.67 min
Batch 5000, Train Loss:0.0171, Train ACC:99.9425
Current time 24.09 min
Batch 5500, Train Loss:0.0169, Train ACC:99.9432
Current time 26.52 min
Batch 6000, Train Loss:0.0169, Train ACC:99.9437
Current time 28.93 min
Batch 6500, Train Loss:0.0168, Train ACC:99.9481
Current time 31.35 min
Batch 7000, Train Loss:0.0170, Train ACC:99.9446
Current tim

In [12]:
pred_vec = [np.array([]) for i in range(len(example_classes))]
Correct,Num = 0,0
for cat in range(len(predict)):
    catname = list(index_classes.keys())[list(index_classes.values()).index(cat)]
    rate = np.sum(np.array([p.argmax() for p in predict[cat]]) == cat)/ len(predict[cat])
    print("{} : {}%, Num : {}".format(catname, rate*100, len(predict[cat])))
    
    Correct =  Correct + np.sum(np.array([p.argmax() for p in predict[cat]]) == cat)
    Num = Num + len(predict[cat])
    
    pred_vec[cat] = np.sum( predict[cat], axis=0 ) / len(predict[cat])

baseball-diamond : 100.0%, Num : 415
basketball-court : 100.0%, Num : 515
bridge : 99.7340425531915%, Num : 1880
ground-track-field : 100.0%, Num : 325
harbor : 99.83255190890824%, Num : 5972
helicopter : 100.0%, Num : 629
large-vehicle : 99.9939893009557%, Num : 16637
plane : 99.98743718592965%, Num : 7960
roundabout : 100.0%, Num : 398
ship : 99.98143541380463%, Num : 26933
small-vehicle : 99.89543955602028%, Num : 24866
soccer-ball-field : 99.68847352024922%, Num : 321
storage-tank : 99.86628036550033%, Num : 4487
swimming-pool : 100.0%, Num : 1731
tennis-court : 99.83058026260059%, Num : 2361


In [25]:
import pandas as pd
pd.DataFrame(pred_vec).to_csv("pred_vec_FCN.csv")